## 5.6 Regresja logistyczna wielu zmiennych
Wiemy już jak zmierzyć jakość modelu klasyfikacji binarnej. Powróćmy więc do przykładu klasyfikacji ryzyka niewypłacalności klientów. Postaramy się zbudować model regresji logistycznej, który będzie szacował takie ryzyko.

Zmierzymy wszystkie wspomniane w poprzednim rozdziale metryki, chociaż niektóre z nich są dla nas bardziej istotne.

In [1]:
import pandas as pd

In [4]:
credit_cards_df = pd.read_parquet("../data/credit_cards_preprocessed.parquet")
credit_cards_df.T

ID,1,2,3,4,5,6,7,8,9,10,...,29991,29992,29993,29994,29995,29996,29997,29998,29999,30000
LIMIT_BAL,20000,120000,90000,50000,50000,50000,500000,100000,140000,20000,...,140000,210000,10000,100000,80000,220000,150000,30000,80000,50000
AGE,24,26,34,37,57,37,29,23,28,35,...,41,34,43,38,34,39,43,37,41,46
PAY_1,2,-1,0,0,-1,0,0,0,0,-2,...,0,3,0,0,2,0,-1,4,1,0
PAY_2,2,2,0,0,0,0,0,-1,0,-2,...,0,2,0,-1,2,0,-1,3,-1,0
PAY_3,-1,0,0,0,-1,0,0,-1,2,-2,...,0,2,0,-1,2,0,-1,2,0,0
PAY_4,-1,0,0,0,0,0,0,0,0,-2,...,0,2,-2,0,2,0,-1,-1,0,0
PAY_5,-2,0,0,0,0,0,0,0,0,-1,...,0,2,-2,0,2,0,0,0,0,0
PAY_6,-2,2,0,0,0,0,0,-1,0,-1,...,0,2,-2,0,2,0,0,0,-1,0
BILL_AMT1,3913,2682,29239,46990,8617,64400,367965,11876,11285,0,...,138325,2500,8802,3042,72557,188948,1683,3565,-1645,47929
BILL_AMT2,3102,1725,14027,48233,5670,57069,412023,380,14096,0,...,137142,2500,10400,1427,77708,192815,1828,3356,78379,48905


### Definicja łańcucha przetwarzania

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

In [6]:
X = credit_cards_df.drop(columns="DEFAULT")
y = credit_cards_df["DEFAULT"]

In [7]:
lr = LogisticRegression(penalty=None, max_iter=10000)
scores = cross_validate(lr, X, y, cv=10, return_train_score=True, scoring=("accuracy", "precision", "recall", "f1"))
scores

/home/maciej/deep/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/maciej/deep/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

{'fit_time': array([18.1743319 , 17.00425935, 30.24855614, 47.35772848, 20.01193976,
        24.43503857, 37.65882087, 37.0401504 , 44.88486695, 19.88063264]),
 'score_time': array([0.00755668, 0.0072186 , 0.01029992, 0.00766182, 0.00902843,
        0.00800133, 0.00798321, 0.00890112, 0.00790644, 0.00808167]),
 'test_accuracy': array([0.79566667, 0.80466667, 0.80166667, 0.80266667, 0.806     ,
        0.80766667, 0.812     , 0.811     , 0.81133333, 0.81733333]),
 'train_accuracy': array([0.80896296, 0.80966667, 0.80748148, 0.80981481, 0.8097037 ,
        0.80722222, 0.80507407, 0.80655556, 0.80507407, 0.80711111]),
 'test_precision': array([0.58252427, 0.621875  , 0.61538462, 0.60909091, 0.62576687,
        0.61417323, 0.66894198, 0.6875    , 0.66440678, 0.70609319]),
 'train_precision': array([0.64587814, 0.64426741, 0.63742898, 0.64356775, 0.64261286,
        0.644881  , 0.64020537, 0.64013453, 0.62438683, 0.63954761]),
 'test_recall': array([0.27108434, 0.2996988 , 0.27710843, 0.302

In [8]:
from sklearn.feature_selection import RFECV

In [13]:
selector = RFECV(lr, step=2, min_features_to_select=1, cv=2, scoring="f1")
selector.fit(X, y)

/home/maciej/deep/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/maciej/deep/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
 

RFECV(cv=2, estimator=LogisticRegression(max_iter=10000, penalty=None),
      scoring='f1', step=2)

In [14]:
selected_features_df = pd.DataFrame({
    "selected": selector.support_,
    "ranking": selector.ranking_
}, index=X.columns)
selected_features_df

,selected,ranking
LIMIT_BAL,True,1
AGE,True,1
PAY_1,True,1
PAY_2,True,1
PAY_3,True,1
PAY_4,True,1
PAY_5,True,1
PAY_6,True,1
BILL_AMT1,True,1
BILL_AMT2,True,1


In [15]:
import numpy as np

In [17]:
np.max(selector.grid_scores_)

AttributeError: 'RFECV' object has no attribute 'grid_scores_'

In [18]:
feature_names = selected_features_df[selected_features_df["selected"] == True].index
feature_names

Index(['LIMIT_BAL', 'AGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
       'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'PAY_AMT1', 'PAY_AMT2',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_OVERDUE_COUNT', 'WEIGHTED_PAYMENT_HISTORY',
       'AVG_PAY_AMT', 'SEX_2', 'EDUCATION_1', 'EDUCATION_2', 'EDUCATION_3',
       'EDUCATION_4', 'EDUCATION_5', 'EDUCATION_6', 'MARRIAGE_1', 'MARRIAGE_2',
       'MARRIAGE_3'],
      dtype='object')

In [19]:
credit_cards_df[feature_names.tolist() + ["DEFAULT"]]

,LIMIT_BAL,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,...,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,EDUCATION_6,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,DEFAULT
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,24,2,2,-1,-1,-2,-2,3913,3102,...,False,True,False,False,False,False,True,False,False,1
2,120000,26,-1,2,0,0,0,2,2682,1725,...,False,True,False,False,False,False,False,True,False,1
3,90000,34,0,0,0,0,0,0,29239,14027,...,False,True,False,False,False,False,False,True,False,0
4,50000,37,0,0,0,0,0,0,46990,48233,...,False,True,False,False,False,False,True,False,False,0
5,50000,57,-1,0,-1,0,0,0,8617,5670,...,False,True,False,False,False,False,True,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,220000,39,0,0,0,0,0,0,188948,192815,...,False,False,True,False,False,False,True,False,False,0
29997,150000,43,-1,-1,-1,-1,0,0,1683,1828,...,False,False,True,False,False,False,False,True,False,0
29998,30000,37,4,3,2,-1,0,0,3565,3356,...,False,True,False,False,False,False,False,True,False,1


In [20]:
credit_cards_df[feature_names.tolist() + ["DEFAULT"]] \
    .to_parquet("../data/credit-cards-reduced.parquet")